# medication

The medications table reflects the active medication orders for patients. These are orders but do not necessarily reflect administration to the patient. For example, while existence of data in the infusionDrug table confirms a patient received a continuous infusion, existence of the same data in this table only indicates that the infusion was *ordered* for the patient. Most orders are fulfilled, but not all. Furthermore, many orders are done *pro re nata*, or PRN, which means "when needed". Administration of these orders is difficult to quantify.

In the US, all orders must be reviewed by a pharmacist. The majority of hospitals have an HL7 medication interface system in place which automatically synchronizes the orders with eCareManager (the source of this database) as they are verified by the pharmacist in the source pharmacy system. For hospitals without a medication interface, the eICU staff may enter a selection of medications to facilitate population management and completeness for reporting purposes.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import holoviews as hv

# for configuring connection 
from configobj import ConfigObj
import os

# holoviews extension can be changed depending on preferences ("plotly", "bokeh, "matplotlib")
hv.extension("plotly") # pip install plotly
%matplotlib inline

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


## Examine a single patient

In [3]:
patientunitstayid = 237395

In [4]:
query = query_schema + """
select *
from medication
where patientunitstayid = {}
order by drugorderoffset
""".format(patientunitstayid)

df = pd.read_sql_query(query, con)
df.head()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\2695020324.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con)


,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,7461175,237395,-12,-91,No,No,VANCOMYCIN 1.5 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,500 3",IV,Once,,No,-92,19
1,10812193,237395,-12,-105,Yes,No,None,4053.0,2000 MG,IV,Once,,No,37,19
2,11612134,237395,-12,-61,Yes,No,None,4034.0,None,IV,Once,,No,247,19
3,11206166,237395,-12,-105,No,No,VANCOMYCIN 1.5 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,500 3",IV,Once,,No,178,19
4,8246015,237395,30,352,Yes,No,None,4053.0,2000 MG,IV,Q8H SCH,,No,3877,19


In [5]:
df.columns

Index(['medicationid', 'patientunitstayid', 'drugorderoffset',
       'drugstartoffset', 'drugivadmixture', 'drugordercancelled', 'drugname',
       'drughiclseqno', 'dosage', 'routeadmin', 'frequency', 'loadingdose',
       'prn', 'drugstopoffset', 'gtc'],
      dtype='object')

In [6]:
# Look at a subset of columns
cols = ['medicationid','patientunitstayid',
        'drugorderoffset','drugorderoffset', 'drugstopoffset',
        'drugivadmixture', 'drugordercancelled', 'drugname','drughiclseqno', 'gtc',
       'dosage','routeadmin','loadingdose', 'prn']
df[cols].head().T

,0,1,2,3,4
medicationid,7461175,10812193,11612134,11206166,8246015
patientunitstayid,237395,237395,237395,237395,237395
drugorderoffset,-12,-12,-12,-12,30
drugorderoffset,-12,-12,-12,-12,30
drugstopoffset,-92,37,247,178,3877
drugivadmixture,No,Yes,Yes,No,Yes
drugordercancelled,No,No,No,No,No
drugname,VANCOMYCIN 1.5 GM IN NS 250 ML IVPB (REPACKAGE),None,None,VANCOMYCIN 1.5 GM IN NS 250 ML IVPB (REPACKAGE),None
drughiclseqno,10093.0,4053.0,4034.0,10093.0,4053.0
gtc,19,19,19,19,19


Here we can see that, roughly on ICU admission, the patient had an order for vancomycin, aztreonam, and tobramycin.

## Identifying patients admitted on a single drug

Let's look for patients who have an order for vancomycin using exact text matching.

In [7]:
drug = 'VANCOMYCIN'
query = query_schema + """
select 
  distinct patientunitstayid
from medication
where drugname like '%{}%'
""".format(drug)

df_drug = pd.read_sql_query(query, con)
print('{} unit stays with {}.'.format(df_drug.shape[0], drug))

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\1557327784.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_drug = pd.read_sql_query(query, con)


20991 unit stays with VANCOMYCIN.


Exact text matching is fairly weak, as there's no systematic reason to prefer upper case or lower case. Let's relax the case matching.

In [8]:
drug = 'VANCOMYCIN'
query = query_schema + """
select 
  distinct patientunitstayid
from medication
where drugname ilike '%{}%'
""".format(drug)

df_drug = pd.read_sql_query(query, con)
print('{} unit stays with {}.'.format(df_drug.shape[0], drug))

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\3871158851.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_drug = pd.read_sql_query(query, con)


30318 unit stays with VANCOMYCIN.


HICL codes are used to group together drugs which have the same underlying ingredient (i.e. most frequently this is used to group brand name drugs with the generic name drugs). We can see above the HICL for vancomycin is 10093, so let's try grabbing that.

In [9]:
hicl = 10093
query = query_schema + """
select 
  distinct patientunitstayid
from medication
where drughiclseqno = {}
""".format(hicl)

df_hicl = pd.read_sql_query(query, con)
print('{} unit stays with HICL = {}.'.format(df_hicl.shape[0], hicl))

9716 unit stays with HICL = 10093.


C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\3971356712.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hicl = pd.read_sql_query(query, con)


No luck! I wonder what we missed? Let's go back to the original query, this time retaining HICL and the name of the drug.

In [10]:
drug = 'VANCOMYCIN'
query = query_schema + """
select 
  drugname, drughiclseqno, count(*) as n
from medication
where drugname ilike '%{}%'
group by drugname, drughiclseqno
order by n desc
""".format(drug)

df_drug = pd.read_sql_query(query, con)
df_drug.head()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\891124085.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_drug = pd.read_sql_query(query, con)


,drugname,drughiclseqno,n
0,VANCOMYCIN HCL 1000 MG IV SOLR,4042.0,9661
1,vancomycin,4042.0,7826
2,VANCOmycin 1 GM in NS 250 mL IVPB,10093.0,3977
3,VANCOMYCIN HCL 10 G IV SOLR,4042.0,3891
4,VANCOMYCIN HCL,4042.0,3064


It appears there are more than one HICL - we can group by HICL in this query to get an idea.

In [11]:
df_drug['drughiclseqno'].value_counts()

drughiclseqno
4042.0     12
8466.0      9
10093.0     5
37442.0     3
Name: count, dtype: int64

Unfortunately, we can't be sure that these HICLs always identify only vancomycin. For example, let's look at drugnames for HICL = 1403.

In [12]:
hicl = 1403
query = query_schema + """
select 
  drugname, count(*) as n
from medication
where drughiclseqno = {}
group by drugname
order by n desc
""".format(hicl)

df_hicl = pd.read_sql_query(query, con)
df_hicl.head()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\2859649644.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hicl = pd.read_sql_query(query, con)


,drugname,n
0,None,200


This HICL seems more focused on the use of creams than on vancomycin. Let's instead inspect the top 3.

In [13]:
for hicl in [4042, 10093, 37442]:
    query = query_schema + """
    select 
      drugname, count(*) as n
    from medication
    where drughiclseqno = {}
    group by drugname
    order by n desc
    """.format(hicl)

    df_hicl = pd.read_sql_query(query, con)
    print('HICL {}'.format(hicl))
    print('Number of rows: {}'.format(df_hicl['n'].sum()))
    print('Top 5 rows by frequency:')
    print(df_hicl.head())
    print()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\3361256922.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hicl = pd.read_sql_query(query, con)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\3361256922.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hicl = pd.read_sql_query(query, con)


HICL 4042
Number of rows: 68951
Top 5 rows by frequency:
                         drugname      n
0                            None  22753
1  VANCOMYCIN HCL 1000 MG IV SOLR   9661
2                      vancomycin   7826
3                        VANCOCIN   4613
4     VANCOMYCIN HCL 10 G IV SOLR   3891

HICL 10093
Number of rows: 18553
Top 5 rows by frequency:
                                           drugname     n
0                                              None  7144
1                 VANCOmycin 1 GM in NS 250 mL IVPB  3977
2  VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE)  2037
3               VANCOmycin 1.25 GM in NS 500ML IVPB  1970
4                VANCOmycin 1.5 GM in NS 500ML IVPB  1944
HICL 37442
Number of rows: 3977
Top 5 rows by frequency:
                                   drugname     n
0                                      None  1977
1  VANCOMYCIN 1 G MINI-BAG PLUS (VIAL MATE)  1856
2                                vancomycin   131
3                                

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\3361256922.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hicl = pd.read_sql_query(query, con)


This is fairly convincing that these only refer to vancomycin. An alternative approach is to acquire the code book for HICL codes and look up vancomycin there.

## Hospitals with data available

In [14]:
query = query_schema + """
with t as
(
select distinct patientunitstayid
from medication
)
select 
  pt.hospitalid
  , count(distinct pt.patientunitstayid) as number_of_patients
  , count(distinct t.patientunitstayid) as number_of_patients_with_tbl
from patient pt
left join t
  on pt.patientunitstayid = t.patientunitstayid
group by pt.hospitalid
""".format(patientunitstayid)

df = pd.read_sql_query(query, con)
df['data completion'] = df['number_of_patients_with_tbl'] / df['number_of_patients'] * 100.0
df.sort_values('number_of_patients_with_tbl', ascending=False, inplace=True)
df.head(n=10)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_35476\1840748422.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con)


,hospitalid,number_of_patients,number_of_patients_with_tbl,data completion
11,73,7059,6836,96.840912
54,167,6092,5825,95.617203
106,264,5237,5111,97.594042
184,420,4679,4618,98.696303
134,338,4277,4241,99.158289
58,176,4328,4186,96.719039
90,243,4243,4172,98.326656
71,199,4240,4116,97.075472
206,458,3701,3624,97.919481
200,443,3656,3612,98.796499


In [15]:
counts, edges = np.histogram(df["data completion"], bins=10)

data_completion_histogram = hv.Histogram((counts, edges))

data_completion_histogram.opts(
    xlabel="Percent of patients with data (binned)",
    ylabel="Number of Records",
    title="Number of Hospitals Data Completion Histogram",
    width=700,
    height=400,
)

:Histogram   [x]   (Frequency)

Here we can see there are a few hospitals with no interface, and thus 0 patients, though the majority have >90% data completion.